### Day One

In [ ]:
# Input

with open("day_one_input.txt") as f:
    data = [int(i) for i in f.read().split("\n")]
print(data[:10])

In [2]:
# Part One

len([True for i in range(len(data)-1) if data[i+1] > data[i]])

1451

In [3]:
# Part Two

len([True for i in range(len(data)) if sum(data[i+1:i+4]) > sum(data[i:i+3])])

1395

### Day Two

In [4]:
# Input

with open("day_two_input.txt") as f:
    data = f.read().split("\n")
print(data[:10])

['forward 4', 'down 9', 'forward 2', 'forward 2', 'down 7', 'up 2', 'down 9', 'up 8', 'down 7', 'down 9']


In [5]:
# Parse data

instructions = [(line.split()[0], int(line.split()[1])) for line in data]

In [6]:
# Part One

horizontal = sum(dist for instr, dist in instructions if instr == "forward")
depth = sum(-dist if instr == "up" else dist if instr == "down" else 0 for instr, dist in instructions)
horizontal*depth

1893605

In [7]:
# Part Two

aim, horizontal, depth = 0, 0, 0
for instr, dist in instructions:
    aim += -dist if instr == "up" else dist if instr == "down" else 0
    if instr == "forward":
        horizontal += dist
        depth += dist*aim
horizontal*depth

2120734350

### Day Three

In [8]:
# Input

with open("day_three_input.txt") as f:
    data = f.read().split("\n")
print(data[:10])

['111111010011', '110011001100', '010011111000', '101001100011', '011011100110', '110100011011', '001000001010', '011011101000', '110011001000', '001011101010']


In [9]:
# Part One

from collections import Counter

gamma, epsilon = [], []
for bit in range(len(data[0])):
    counts = Counter([diagnostic[bit] for diagnostic in data]).most_common()
    gamma.append(counts[0][0])
    epsilon.append(counts[-1][0])
int("".join(gamma), 2)*int("".join(epsilon), 2)

3985686

In [10]:
# Part Two

ratings = []
for counter_index, bit_value in zip((0, -1), ("1", "0")):
    data_subset = data
    bit_index = 0
    while len(data_subset) > 1:
        counts = Counter([diagnostic[bit_index] for diagnostic in data_subset]).most_common()
        bit_test = bit_value if counts[0][1] == counts[1][1] else counts[counter_index][0]
        data_subset = [diagnostic for diagnostic in data_subset if diagnostic[bit_index] == bit_test]
        bit_index += 1
    ratings.append(int("".join(data_subset[0]), 2))
ratings[0]*ratings[1]

2555739

### Day Four

In [132]:
# Input

with open("day_four_input.txt") as f:
    numbers = f.readline().strip()
numbers = [int(n) for n in numbers.split(",")]
boards = np.loadtxt("day_four_input.txt", skiprows=1)
boards = np.stack(boards)
boards = np.vsplit(boards, boards.shape[0]/5)
print(numbers[:10])
print(boards[0])

[49, 48, 98, 84, 71, 59, 37, 36, 6, 21]
[[86. 46. 47. 61. 57.]
 [44. 74. 17.  5. 87.]
 [78.  8. 54. 55. 97.]
 [11. 90.  7. 75. 70.]
 [81. 50. 84. 10. 60.]]


In [138]:
# Part One

results = []
for i, board in enumerate(boards):
    mask = np.full((5, 5), False)
    numbers_iter = iter(numbers)
    while 5 not in np.sum(mask, axis=0) and 5 not in np.sum(mask, axis=1):
        last_called = next(numbers_iter)
        mask = mask | (board == last_called)
    results.append({
        "board": i,
        "sum": int(last_called*board[~mask].sum()),
        "order": numbers.index(last_called)
    })

sorted(results, key=lambda k: k["order"])[0]["sum"]

65325

In [139]:
# Part Two

sorted(results, key=lambda k: k["order"], reverse=True)[0]["sum"]

4624